# Notebook for the development of module "weather -> photovoltaic output"
### If you have to develop some new function useful also in other situations, please put it in a dedicated .py file, then import it here

In [ ]:
import numpy as np        #pip install numpy
import pvlib              #pip install pvlib            #library to automatically convert weather params into pv output